In [0]:
# !apt -y install libcusparse.so.8.0 libnvrtc.so.8.0 libnvtoolsext1
# !ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
# !pip install cupy-cuda100

In [0]:
!curl https://colab.chainer.org/install | sh -

In [0]:
!chainer.print_runtime_info()

In [0]:
import cupy as cp
import matplotlib.pyplot as plt
from termcolor import colored
from six.moves import cPickle

In [0]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

dir_path   = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
/class-MachineLearning/assignment11/movie_review"
review_data = load_files(dir_path)
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [0]:
tr_x = cp.array(list(X_train)).T
ts_x = cp.array(list(X_test)).T
tr_y = cp.array(list(y_train)).T
ts_y = cp.array(list(y_test)).T
print(tr_x.shape)
print(ts_x.shape)
print(tr_y.shape)
print(ts_y.shape)

In [0]:
cupy_pickle=[]
cupy_pickle.append(tr_x)
cupy_pickle.append(ts_x)
cupy_pickle.append(tr_y)
cupy_pickle.append(ts_y)

# Save pickles

In [0]:
# pickle_path = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
# /class-MachineLearning/assignment11/cp_pickles"
# cPickle.dump(cupy_pickle, open(pickle_path, "wb", True))

# Load pickles

In [0]:
pickle_path = "drive/My Drive/Classroom/Machine Learning (2) 2020-1\
/class-MachineLearning/assignment11/cp_pickles"
load_pickles = cPickle.load(open(pickle_path, "rb", True))
print(len(load_pickles))

In [0]:
train_x, test_x, train_y, test_y = load_pickles
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)
print(type(train_x))
one_hot_train=cp.expand_dims(train_y, axis=0)
one_hot_test=cp.expand_dims(test_y, axis=0)

In [0]:
'''
Config
'''


batch_size=100

num_train = train_x.shape[1]
num_test = test_x.shape[1]

one_hot_train=cp.expand_dims(train_y, axis=0)
one_hot_test=cp.expand_dims(test_y, axis=0)

layer1=cp.empty((800, num_train), dtype=float)
layer2=cp.empty((1, num_train), dtype=float)
layers=[train_x, layer1, layer2]


std_w0 = cp.sqrt(2/2300)
weight0=cp.random.normal(0., std_w0, (800, train_x.shape[0]))
std_w1 = cp.sqrt(2/801)
weight1=cp.random.normal(0., std_w1, (1, 800))
weights=[weight0, weight1]

# m_t=[np.zeros((196, 784)), np.zeros((49, 196)), np.zeros((10, 49))]
# v_t=[np.zeros((196, 784)), np.zeros((49, 196)), np.zeros((10, 49))]

m_t=[cp.zeros((800, train_x.shape[0])),cp.zeros((1, 800))]
v_t=[cp.zeros((800, train_x.shape[0])),cp.zeros((1, 800))]

train_loss=[]
train_accuracy=[]

test_loss=[]
test_accuracy=[]

print(num_train)
print(num_test)

In [0]:
do=[0, 1]
p_do=[0, 1]
def sigmoid(x) :
    return 1/(1+cp.exp(-x))

def fw_propagation() :
    #a_list=[]
    #input_data = np.vstack(((np.full((1, list_image.shape[1]), bias0)), list_image))
    for i in range(len(weights)-1) :
        #print('a'*50)
        do[i] = cp.random.binomial(1, 0.9, size=(layers[i+1][:,0].shape[0], 1))
        #print('-'*50)
        p_do[i] = float(cp.count_nonzero(do[i])*1000 / (layers[i+1].shape[0] * layers[i+1].shape[1]))
        #print('*'*50)
    # do[0]=np.random.binomial(1, 0.7, size=layers[1].shape)
    # do[1]=np.random.binomial(1, 0.7, size=layers[2].shape)
    # do[2]=np.random.binomial(1, 0.7, size=layers[3].shape)
    for i in range(len(weights)) :
        layers[i+1]=sigmoid(weights[i] @ layers[i]) * do[i]
#        layers[i+1]=sigmoid(weights[i] @ layers[i])



def predict(train=False) :
    if (not train) :
        for i in range(len(weights)) :
            if i==0 : 
                output = sigmoid(weights[i] @ test_x)
                
            elif i==len(weights)-1 : 
                output = sigmoid(weights[i] @ output)
            else :
                output = sigmoid(weights[i] @ output)

    else : 
        for i in range(len(weights)) :
            if i==0 : 
                output = sigmoid(weights[i] @ train_x) * p_do[i]

            elif i==len(weights)-1 : 
                output = sigmoid(weights[i] @ output)
            else :
                output = sigmoid(weights[i] @ output)  * p_do[i]

    return output

In [0]:

'''
Back propagation
'''
learning_rate=0.001
lbd = 0
beta1, beta2 = 0.9, 0.999
eps=1e-8
t=0
gam=0.9
def bw_propagation() :
    global t, m_t, v_t, eps, beta1, beta2
    dev_h=layers[-1] - one_hot_train
    t+=1
    for i in range(len(weights)-1, -1, -1) :
        dev_w=dev_h @ layers[i].T / num_train
        
        if i==0 : 
            pass
        else :
            #dev_h=weights[i].T @ dev_h * layers[i] * (1-layers[i]) * do[i-1]
            dev_h=weights[i].T @ dev_h * layers[i] * (1-layers[i])
        
        
        #weights[i] -= (learning_rate*m_hat)/(cp.sqrt(v_hat) + eps) + lbd * weights[i]
        #weights[i] -= (learning_rate*m_hat)/(cp.sqrt(v_hat) + eps) + learning_rate * lbd * weights[i]
        ############
        
        # m_t[i] = beta1*m_t[i] + (1-beta1)*dev_w 
        # v_t[i] = beta2*v_t[i] + (1-beta2)*dev_w*dev_w
        # m_hat=m_t[i]/(1-(beta1**t))
        # v_hat=v_t[i]/(1-(beta2**t))
        # weights[i] -= (learning_rate*m_hat)/(cp.sqrt(v_hat) + eps)+ learning_rate * lbd * weights[i]
        ############
        
        if m_t[i].sum ==0 : m_t[i] += dev_w*dev_w
        else : m_t[i]=gam*m_t[i] + (1-gam) * dev_w*dev_w
        weights[i] -= learning_rate * dev_w /(cp.sqrt(m_t[i] + eps))+ learning_rate * lbd * weights[i]
        #weights[i] -= learning_rate * dev_w /(cp.sqrt(m_t[i] + eps))

def loss(output, label) :
    return cp.mean(cp.sum(-label*cp.log(output) - (1-label)*cp.log(1-output), axis=0))
# def accuracy(output, label):
#   return (output == label.argmax(axis = 0)).mean()*100
def accuracy(output, label) :
    m=label.shape[1]
    over_p=cp.array(output>=0.5, dtype=cp.float64) 
    over_c=cp.array(label==over_p, dtype=cp.float64) 
    # print(over_p.shape)
    # print(label.shape)
    # print(over_c)
    return (cp.sum(over_c)/m)*100

In [0]:

# '''
# Back propagation
# '''
# lbd = 0.25
# beta1, beta2 = 0.9, 0.999
# eps=1e-8
# t=0
# def bw_propagation() :
#     global t, m_t, v_t, eps, beta1, beta2
#     dev_h=layers[-1] - one_hot_train
#     t+=1
#     for i in range(len(weights)-1, -1, -1) :
#         dev_w=dev_h @ layers[i].T
#         if i==0 : 
#             pass
#         else :
#             #dev_h=weights[i].T @ dev_h * layers[i] * (1-layers[i]) * do[i-1]
#             dev_h=weights[i].T @ dev_h * layers[i] * (1-layers[i])
        
#         m_t[i] = beta1*m_t[i] + (1-beta1)*dev_w / num_train
#         v_t[i] = beta2*v_t[i] + (1-beta2)*cp.power((dev_w/num_train), 2)
#         m_hat=m_t[i]/(1-(beta1**t))
#         v_hat=v_t[i]/(1-(beta2**t))
#         #weights[i] -= (learning_rate*m_hat)/(cp.sqrt(v_hat) + eps) + lbd * weights[i]
#         weights[i] -= (learning_rate*m_hat)/(cp.sqrt(v_hat) + eps) + learning_rate * lbd * weights[i]
#         #weights[i] -= learning_rate * dev_w / num_train

# def loss(output, label) :
#     return cp.mean(cp.sum(-label*cp.log(output) - (1-label)*cp.log(1-output), axis=0))
# # def accuracy(output, label):
# #   return (output == label.argmax(axis = 0)).mean()*100
# def accuracy(output, label) :
#     m=label.shape[1]
#     over_p=cp.array(output>=0.5, dtype=cp.float64) 
#     over_c=cp.array(label==over_p, dtype=cp.float64) 
#     # print(over_p.shape)
#     # print(label.shape)
#     # print(over_c)
#     return (cp.sum(over_c)/m)*100

In [0]:
import time
start = time.time() 
#learning_rate=0.001
for i in range(1000) :
    # input_data, a0, a1, a2 = fw_propagation()
    if i%100==0 :
        print("time :", time.time() - start)
    fw_propagation()
    bw_propagation()
    p = predict(False)
    p_t = predict(True)
    train_loss.append(loss(p_t, one_hot_train))
    test_loss.append(loss(p, one_hot_test))
    train_accuracy.append(accuracy(p_t, one_hot_train))
    test_accuracy.append(accuracy(p, one_hot_test))
    if float(test_accuracy[-1]) >= 84 :
        learning_rate=1e-20


print("time :", time.time() - start)

In [0]:
'''
Visualize Loss
'''
plt.figure(figsize=(8, 5))
plt.plot(train_loss, color='blue', label='train')
plt.plot(test_loss, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("cost")
plt.legend(loc='upper right')
plt.title("Loss")
plt.show()

In [0]:
'''
Visualize accuracy
'''
plt.figure(figsize=(8, 5))
plt.plot(train_accuracy, color='blue', label='train')
plt.plot(test_accuracy, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

In [0]:
print(test_accuracy[-10:])

In [0]:
print(max(test_accuracy))
print(test_accuracy.index(max(test_accuracy)))
print(learning_rate)

# 1. Plot the loss curve

---
\
     
* train loss는 파란색, test loss는 빨간색으로 plot했습니다.

In [0]:
'''
Visualize Loss
'''
plt.figure(figsize=(8, 5))
plt.plot(train_loss, color='blue', label='train')
plt.plot(test_loss, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("cost")
plt.legend(loc='upper right')
plt.title("Loss")
plt.show()

# 2. Plot the accuracy curve

---
\
     
* train accuracy는 파란색, test accuracy는 빨간색으로 plot했습니다.

In [0]:
'''
Visualize accuracy
'''
plt.figure(figsize=(8, 5))
plt.plot(train_accuracy, color='blue', label='train')
plt.plot(test_accuracy, color='red', label='test')
plt.xlabel("iteration")
plt.ylabel("accuracy")
plt.legend(loc='upper right')
plt.title("Accuracy")
plt.show()

# 3. Plot the quantitative results

---

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
p_train=predict(train=True).flatten()
p_test=predict(train=False).flatten()
p_train=p_train>=0.5
p_test=p_test>=0.5
print(confusion_matrix(train_y.tolist(),p_train.tolist()))
print(classification_report(train_y.tolist(),p_train.tolist()))
print(accuracy_score(train_y.tolist(),p_train.tolist()))

print(confusion_matrix(test_y.tolist(),p_test.tolist()))
print(classification_report(test_y.tolist(),p_test.tolist()))
print(accuracy_score(test_y.tolist(),p_test.tolist()))


# 4. Plot the accuracy value

---

In [0]:
print("<Final accuracy>")
print('-'*50)
train_final='train accuracy : '+str(train_accuracy[-1]) + '%'
test_final='test accuracy : '+str(test_accuracy[-1]) + '%'
#print(colored(train_final, 'blue'))
print(colored(test_final, 'red'))
print('-'*50)